# ETL Pipeline

In [1]:
import numpy as np 
import pandas as pd 
import re 
from typing import List
from datetime import datetime

# adjust pandas display settings
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 500)


import os
import mysql.connector as sql

In [2]:
# count the number of excel sheets
xl_file = pd.ExcelFile("./data/world population.xlsx")
num_of_sheets = len(xl_file.sheet_names)
num_of_sheets

11

### Load the data
* The population is in **thousands**

In [3]:
# Load data
xl_data_1 = pd.read_excel("./data/world population.xlsx", sheet_name=0, header=16)

xl_data_1.head(20)

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,Estimates,WORLD,NaN,900,World,0,2.53643e+06,2.58403e+06,2.63086e+06,2.67761e+06,2.72485e+06,2.77302e+06,2.82244e+06,2.87331e+06,2.92569e+06,2.97958e+06,3.03495e+06,3.09184e+06,3.15042e+06,3.211e+06,3.27398e+06,3.33958e+06,3.40792e+06,3.47877e+06,3.5516e+06,3.62568e+06,3.70044e+06,3.77576e+06,3.85165e+06,3.92778e+06,4.00379e+06,4.07948e+06,4.15467e+06,4.22951e+06,4.30453e+06,4.38051e+06,4.458e+06,4.537e+06,4.61739e+06,4.69957e+06,4.78401e+06,4.87092e+06,4960568,5.05252e+06,5.14543e+06,5.23744e+06,5.32723e+06,5.41429e+06,5.49892e+06,5.5816e+06,5.66315e+06,5.74421e+06,5.82489e+06,5.90505e+06,5.98479e+06,6.06424e+06,6.14349e+06,6.22263e+06,6.30177e+06,6.38119e+06,6.46116e+06,6.54191e+06,6.62352e+06,6.70595e+06,6.78909e+06,6.87277e+06,6.95682e+06,7.04119e+06,7.12583e+06,7.21058e+06,7.29529e+06,7.3798e+06,7.46402e+06,7.54786e+06,7.63109e+06,7.71347e+06,7.7948e+06
1,2,Estimates,UN development groups,a,1803,Label/Separator,900,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,3,Estimates,More developed regions,b,901,Development Group,1803,814819,824004,833720,843788,854060,864430,874824,885207,895576,905949,916344,926752,937111,947303,957175,966624,975585,984097,992293,1.00037e+06,1.00845e+06,1.0166e+06,1.02476e+06,1.03286e+06,1.04077e+06,1.04842e+06,1.05577e+06,1.06288e+06,1.06978e+06,1.07651e+06,1.08313e+06,1.08961e+06,1.09596e+06,1.10223e+06,1.10848e+06,1.11476e+06,1.1211e+06,1.12745e+06,1.13373e+06,1.13978e+06,1.14551e+06,1.15089e+06,1.15597e+06,1.16074e+06,1.16523e+06,1.16948e+06,1.17349e+06,1.17728e+06,1.18095e+06,1.18462e+06,1.18836e+06,1.1922e+06,1.19616e+06,1.20027e+06,1.20461e+06,1.20922e+06,1.21411e+06,1.21925e+06,1.22452e+06,1.22974e+06,1.23477e+06,1.23956e+06,1.24411e+06,1.24845e+06,1.25262e+06,1.25662e+06,1.26048e+06,1.26415e+06,1.26756e+06,1.27063e+06,1.2733e+06
3,4,Estimates,Less developed regions,c,902,Development Group,1803,1.72161e+06,1.76003e+06,1.79714e+06,1.83382e+06,1.87079e+06,1.90859e+06,1.94762e+06,1.9881e+06,2.03011e+06,2.07363e+06,2.11861e+06,2.16509e+06,2.21331e+06,2.2637e+06,2.3168e+06,2.37296e+06,2.43234e+06,2.49467e+06,2.55931e+06,2.62532e+06,2.69199e+06,2.75916e+06,2.82689e+06,2.89492e+06,2.96302e+06,3.03106e+06,3.09889e+06,3.16662e+06,3.23475e+06,3.30399e+06,3.37488e+06,3.44739e+06,3.52143e+06,3.59734e+06,3.67553e+06,3.75616e+06,3.83947e+06,3.92507e+06,4.0117e+06,4.09766e+06,4.18172e+06,4.2634e+06,4.34295e+06,4.42086e+06,4.49792e+06,4.57473e+06,4.6514e+06,4.72776e+06,4.80384e+06,4.87962e+06,4.95513e+06,5.03042e+06,5.10562e+06,5.18091e+06,5.25655e+06,5.33269e+06,5.40941e+06,5.4867e+06,5.56457e+06,5.64303e+06,5.72206e+06,5.80164e+06,5.88171e+06,5.96213e+06,6.04268e+06,6.12317e+06,6.20354e+06,6.28371e+06,6.36353e+06,6.44284e+06,6.52149e+06
4,5,Estimates,Least developed countries,d,941,Development Group,902,195428,199180,203015,206986,211133,215486,220061,224862,229882,235106,240518,246114,251901,257908,264174,270721,277573,284709,292046,299474,306917,314345,321798,329358,337143,345243,353691,362475,371577,380963,390613,400529,410741,421279,432185,443491,455194,467290,479814,492803,506276,520262,534731,549559,564581,579682,594801,609983,625342,641047,657216,673903,691061,708588,726331,744191,762133,780220,798563,817323,836615,856471,876867,897793,919223,941131,963520,986385,1.00969e+06,1.03339e+06,1.05744e+06
5,6,Estimates,"Less deve

### Tidy the data i.e Convert from wide format to long format

In [4]:
population = (
        pd.melt(xl_data_1, 
                id_vars=['Index', 'Variant', 'Region, subregion, country or area *', 'Notes', 'Country code', 'Type', 'Parent code'],
                var_name='Years',
                value_name='Population(Thousands)')  
    )

population

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,Years,Population(Thousands)
0,1,Estimates,WORLD,NaN,900,World,0,1950,2.53643e+06
1,2,Estimates,UN development groups,a,1803,Label/Separator,900,1950,...
2,3,Estimates,More developed regions,b,901,Development Group,1803,1950,814819
3,4,Estimates,Less developed regions,c,902,Development Group,1803,1950,1.72161e+06
4,5,Estimates,Least developed countries,d,941,Development Group,902,1950,195428
...,...,...,...,...,...,...,...,...,...
20514,285,Estimates,Bermuda,14,60,Country/Area,918,2020,62.273
20515,286,Estimates,Canada,NaN,124,Country/Area,918,2020,37742.2
20516,287,Estimates,Greenland,26,304,Country/Area,918,2020,56.772
20517,288,Estimates,Saint Pierre and Miquelon,2,666,Country/Area,918,2020,5.795


### Clean the data

In [5]:
population = population.copy()
# replace 'NaNs' with 'null'
population['Notes'] = population['Notes'].replace(np.nan, '')

# convert to string
population['Notes'] = population['Notes'].astype('string')

In [6]:
# filter out Groups, Continents and Regions
region_population = population.loc[population['Notes'].str.lower().str.contains(r"[a-p]")]

region_population = region_population.copy()
# rename column
region_population.rename(columns={'Region, subregion, country or area *': 'Region'}, inplace=True)
# convert to datetime
region_population['Years'] = pd.to_datetime(region_population['Years'])
# # filter out the year, month and day
# region_population['Years'] = region_population['Years'].apply(year_month)
# replace '...' with '0'
region_population['Population(Thousands)'] = region_population['Population(Thousands)'].replace('...', 0)

region_population.head()

,Index,Variant,Region,Notes,Country code,Type,Parent code,Years,Population(Thousands)
1,2,Estimates,UN development groups,a,1803,Label/Separator,900,1950-01-01,0.000
2,3,Estimates,More developed regions,b,901,Development Group,1803,1950-01-01,814818.913
3,4,Estimates,Less developed regions,c,902,Development Group,1803,1950-01-01,1721612.105
4,5,Estimates,Least developed countries,d,941,Development Group,902,1950-01-01,195427.785
5,6,Estimates,"Less developed regions, excluding least develo...",e,934,Development Group,902,1950-01-01,1526184.320


In [7]:
# save as csv
region_population.to_csv('./data/region_population.csv', index=False)

In [8]:
# Add all the rows to a list
val_reg_pop = []
for row in region_population.itertuples():
    val_reg_pop.append(row[1:])

# view the first 3 elements    
val_reg_pop[:3]

[(2,
  'Estimates',
  'UN development groups',
  'a',
  1803,
  'Label/Separator',
  900,
  Timestamp('1950-01-01 00:00:00'),
  0.0),
 (3,
  'Estimates',
  'More developed regions',
  'b',
  901,
  'Development Group',
  1803,
  Timestamp('1950-01-01 00:00:00'),
  814818.913),
 (4,
  'Estimates',
  'Less developed regions',
  'c',
  902,
  'Development Group',
  1803,
  Timestamp('1950-01-01 00:00:00'),
  1721612.105)]

In [9]:
# filter out Countries
countries_population = population.loc[population['Notes'].str.lower().str.contains(r"[^a-p]")]

countries_population = countries_population.copy()
# rename column
countries_population.rename(columns={'Region, subregion, country or area *': 'Country'}, inplace=True)
# convert to datetime
countries_population['Years'] = pd.to_datetime(countries_population['Years'])

# convert to numeric data type
countries_population['Population(Thousands)'] = pd.to_numeric(countries_population['Population(Thousands)'])

# drop column
countries_population = countries_population.drop(columns=['Notes'])

countries_population.head()

,Index,Variant,Country,Country code,Type,Parent code,Years,Population(Thousands)
34,35,Estimates,Mauritius,480,Country/Area,910,1950-01-01,493.258
35,36,Estimates,Mayotte,175,Country/Area,910,1950-01-01,15.138
37,38,Estimates,Réunion,638,Country/Area,910,1950-01-01,248.114
43,44,Estimates,United Republic of Tanzania,834,Country/Area,910,1950-01-01,7649.765
76,77,Estimates,Saint Helena,654,Country/Area,914,1950-01-01,5.249


In [10]:
# save as csv
countries_population.to_csv('./data/countries_population.csv', index=False)

In [11]:
# Add all the rows to a list
val_ctry_pop = []
for row in countries_population.itertuples():
    val_ctry_pop.append(row[1:])

# view the first 3 elements    
val_ctry_pop[:3]

[(35,
  'Estimates',
  'Mauritius',
  480,
  'Country/Area',
  910,
  Timestamp('1950-01-01 00:00:00'),
  493.258),
 (36,
  'Estimates',
  'Mayotte',
  175,
  'Country/Area',
  910,
  Timestamp('1950-01-01 00:00:00'),
  15.138),
 (38,
  'Estimates',
  'Réunion',
  638,
  'Country/Area',
  910,
  Timestamp('1950-01-01 00:00:00'),
  248.114)]

In [12]:
### Helper Functions
def create_connection():
    """Create a MySQL connection and return the connection. It returns db."""
    config = {
        'host': 'localhost',
        'user': os.environ.get('DB_USER'),
        'password': os.environ.get('DB_PSWD')
    }
    # create connection
    db = sql.connect(**config)
    return db


def create_db(DB_NAME, sql_query):
    """Create a MySQL DB"""
    cursor.execute(sql_query)
    print(f"{DB_NAME} was created")
    

def create_db_connection(DB_NAME):
    """Connect the MySQL DB with the connection and return the connection. It returns db."""
    # create connection
    db = sql.connect(
        host = 'localhost',
        user = os.environ.get('DB_USER'),
        password = os.environ.get('DB_PSWD'),
        database = DB_NAME
    )
    return db


def create_table(table_name, sql_query):
    """Add a table to the DB"""
    # execute the query
    cursor.execute(sql_query)
    print(f"{table_name} was created")
    

def show_tables():
    # check for the tables
    cursor.execute('SHOW TABLES')
    # display tables
    for table in cursor:
        print(table)
        
        
def insert_into_table(table_name, sql_query, values):
    """Insert values into a table"""
    # add all the rows to the db
    cursor.executemany(sql_query, values)
    
    # commit the changes
    db.commit()
    print(f"Values inserted into {table_name}")
    
    
def select_records(sql_query):
    """Select records from a table"""
    cursor.execute(sql_query)
    # display records
    for x in cursor:
        print(f"{x} \n")

<hr>

### ETL Pipeline
* Create connection
* Create DB
* Create tables
* Populate tables

In [13]:
# Create connection
db = create_connection()

# create cursor object using the connection
cursor = db.cursor()

# Create DB
DB_NAME = 'World_population'
sql_query = f"CREATE SCHEMA IF NOT EXISTS {DB_NAME}"
create_db(DB_NAME, sql_query)

# create connection with DB
db = create_db_connection(DB_NAME)

# create cursor object using the connection
cursor = db.cursor()

World_population was created


In [14]:
# create a table Region_Population
table_name = 'Region_Population'
sql_query = f"""
                CREATE TABLE IF NOT EXISTS {table_name}
                    (
                        ID INT NOT NULL,
                        Variant VARCHAR(100),
                        Region VARCHAR(100),
                        Notes VARCHAR(100),
                        Country_code INT,
                        Type VARCHAR(100),
                        Parent_code INT,
                        Years DATE,
                        Population_Thousands INT                    
                    )
            """


create_table(table_name, sql_query)

# query
sql_query = f"""
                INSERT INTO {table_name}
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """ 

# populate table
insert_into_table(table_name, sql_query, val_reg_pop)

Region_Population was created
Values inserted into Region_Population


In [15]:
# create a table Countries_population
table_name = 'Countries_population'
sql_query = f"""
                CREATE TABLE IF NOT EXISTS {table_name}
                    (
                        ID INT NOT NULL,
                        Variant VARCHAR(100),
                        Country VARCHAR(100),
                        Country_code INT,
                        Type VARCHAR(100),
                        Parent_code INT,
                        Years DATE,
                        Population_Thousands INT                    
                    )
            """

# create a table Countries_population
create_table(table_name, sql_query)

sql_query = f"""
                INSERT INTO {table_name}
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """ 

# populate table
insert_into_table(table_name, sql_query, val_ctry_pop)

# show tables
show_tables()

Countries_population was created
Values inserted into Countries_population
('countries_population',)
('region_population',)


In [16]:
# select some records
sql_query = """
                SELECT * FROM region_population
                    LIMIT 5
            """
select_records(sql_query)

# close connection
db.close()

(2, 'Estimates', 'UN development groups', 'a', 1803, 'Label/Separator', 900, datetime.date(1950, 1, 1), 0) 

(3, 'Estimates', 'More developed regions', 'b', 901, 'Development Group', 1803, datetime.date(1950, 1, 1), 814819) 

(4, 'Estimates', 'Less developed regions', 'c', 902, 'Development Group', 1803, datetime.date(1950, 1, 1), 1721612) 

(5, 'Estimates', 'Least developed countries', 'd', 941, 'Development Group', 902, datetime.date(1950, 1, 1), 195428) 

(6, 'Estimates', 'Less developed regions, excluding least developed countries', 'e', 934, 'Development Group', 902, datetime.date(1950, 1, 1), 1526184) 



In [ ]:
# Load data
xl_data_2 = pd.read_excel("./data/world population.xlsx", sheet_name=1, header=16)

xl_data_2.head(20)